In [1]:
import config_readwrite as crw
import numpy as np
import os, sys
import pandas as pd
import pybedtools as pbt
from scipy import stats

LOCAL = False

In [2]:
if LOCAL is True:
    PATH = "/Users/sarahfong/Desktop/local_data/Biomarin/results/plaq-seq_hg19/"
    config_name = os.path.join()
else: 
    PATH = "/wynton/home/ahituv/fongsl/biomarin/data/plac-seq/"
    config_name = "/wynton/home/ahituv/fongsl/biomarin/bin/config.neuron.mpra.orig.ini"

In [3]:
config, cfn = crw.read(config_name)

In [13]:
#ENH_BED = config["lib2"]['enh_bed']
ENH_BED = '/wynton/group/ahituv/fongsl/projects/biomarin/library_2/design/lib2.merged.tiles.bed'

NOTT_BED="/wynton/home/ahituv/fongsl/biomarin/data/plac-seq/Nott_2019_plac_seq_elements_hg19.liftOver.to.Hg38.bed"
MPRA = "/wynton/group/ahituv/fongsl/projects/biomarin/data/deltaMPRA.gaba.minus.glut.tsv"
META = '/wynton/group/ahituv/fongsl/projects/biomarin/data/lib2.processed.bed'

In [5]:
enh = pbt.BedTool(ENH_BED)
plac = pbt.BedTool(NOTT_BED)

In [6]:
a = enh.intersect(plac, wao = True)
v = 0
for i in a:
    v+=1
print(v)

80428


## read intersection

In [27]:
df = pd.read_csv(a.fn,sep='\t', header=None).drop_duplicates()
df.columns=["#chr", "start", "end", 
            "#chr_enh", "start_enh", "end_enh", 
            "#chr_plac", "start_plac", "end_plac", 
            "plac.id", "plac.celltypes", "plac.datatype",
            "overlap"
           ]
df.head()

,#chr,start,end,#chr_enh,start_enh,end_enh,#chr_plac,start_plac,end_plac,plac.id,plac.celltypes,plac.datatype,overlap
0,chr1,10057,10327,chr1,10057,10563,.,-1,-1,.,.,.,0
1,chr1,10077,10347,chr1,10057,10563,.,-1,-1,.,.,.,0
2,chr1,10097,10367,chr1,10057,10563,.,-1,-1,.,.,.,0
3,chr1,10117,10387,chr1,10057,10563,.,-1,-1,.,.,.,0
4,chr1,10133,10403,chr1,10057,10563,.,-1,-1,.,.,.,0


## open MPRA

In [12]:
mpra = pd.read_csv(MPRA, sep='\t')
mpra

,name,gaba-label,glut-label,celltype_dif,gaba,glut,delta.gaba-glut
0,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",silencing,False,True,-1.817551,-0.677805,-1.139746
1,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",silencing,False,True,-1.806478,-0.697447,-1.109031
2,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",False,False,False,-1.492956,-0.743462,-0.749494
3,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",silencing,False,True,-1.794754,-0.614067,-1.180686
4,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",silencing,silencing,False,-1.856817,-1.041096,-0.815721
...,...,...,...,...,...,...,...
35366,shuffle_94_chr15:32646056-32646326,False,False,False,0.240804,0.097280,0.143524
35367,shuffle_95_chr19:13408968-13409238,False,False,False,0.223373,-0.022828,0.246201
35368,shuffle_96_chr13:58618367-58618637,False,False,False,0.178189,0.096863,0.081326
35369,shuffle_98_chr3:12869018-12869288,False,False,False,-0.225030,-0.183407,-0.041623


In [57]:
meta = pd.read_csv(META, sep='\t')
meta = meta.loc[~meta["#chr"].str.contains("seq")] # remove background sequences
print(meta.shape)
meta.head()

(42722, 4)


,#chr,start,end,name
648,chr1,10057.0,10327.0,chr1:10057-10327
649,chr1,10077.0,10347.0,chr1:10077-10347
650,chr1,10097.0,10367.0,chr1:10097-10367
651,chr1,10117.0,10387.0,chr1:10117-10387
652,chr1,10133.0,10403.0,chr1:10133-10403


In [58]:
mpra = pd.merge(meta, mpra) # add meta data

mpra.head()

,#chr,start,end,name,gaba-label,glut-label,celltype_dif,gaba,glut,delta.gaba-glut
0,chr1,1692002.0,1692272.0,chr1:1692002-1692272,False,False,False,0.525588,0.205703,0.319886
1,chr1,1692022.0,1692292.0,chr1:1692022-1692292,False,False,False,0.602023,0.072854,0.529169
2,chr1,1692042.0,1692312.0,chr1:1692042-1692312,False,False,False,0.532336,0.115079,0.417256
3,chr1,1692062.0,1692332.0,chr1:1692062-1692332,False,False,False,0.217245,-0.091700,0.308945
4,chr1,1692082.0,1692352.0,chr1:1692082-1692352,False,False,False,0.265050,0.096656,0.168395


In [61]:
df = pd.merge(meta, df) # add meta data

## separate overlap and no overlap

In [65]:
no_overlap = df.loc[df["overlap"]==0].copy()
overlap = df.loc[df["overlap"]>0].copy() # get only the elements that overlap HiC elements

no_overlap

,#chr,start,end,name,#chr_enh,start_enh,end_enh,#chr_plac,start_plac,end_plac,plac.id,plac.celltypes,plac.datatype,overlap
0,chr1,10057.0,10327.0,chr1:10057-10327,chr1,10057,10563,.,-1,-1,.,.,.,0
1,chr1,10077.0,10347.0,chr1:10077-10347,chr1,10057,10563,.,-1,-1,.,.,.,0
2,chr1,10097.0,10367.0,chr1:10097-10367,chr1,10057,10563,.,-1,-1,.,.,.,0
3,chr1,10117.0,10387.0,chr1:10117-10387,chr1,10057,10563,.,-1,-1,.,.,.,0
4,chr1,10133.0,10403.0,chr1:10133-10403,chr1,10057,10563,.,-1,-1,.,.,.,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80730,chrY,12742217.0,12742487.0,chrY:12742217-12742487,chrY,12741976,12742567,.,-1,-1,.,.,.,0
80731,chrY,12742237.0,12742507.0,chrY:12742237-12742507,chrY,12741976,12742567,.,-1,-1,.,.,.,0
80732,chrY,12742257.0,12742527.0,chrY:12742257-12742527,chrY,12741976,12742567,.,-1,-1,.,.,.,0
80733,chrY,12742277.0,12742547.0,chrY:12742277-12742547,chrY,12741976,12742567,.,-1,-1,.,.,.,0


In [35]:
overlap.groupby(["plac.datatype", "plac.celltypes"])["#chr"].count()

plac.datatype  plac.celltypes                    
e              Astrocyte                              5409
               Microglia                              2427
               Neuronal                               9727
               Oligo                                  5372
h              Microglia                              4133
               Microglia,Neuronal                     3828
               Microglia,Neuronal,Oligo               7352
               Microglia,Oligo                        1338
               Microglia,Oligo,Neuronal                876
               Neuronal                               4486
               Neuronal,Microglia                      615
               Neuronal,Microglia,Oligo                592
               Neuronal,Oligo                         2244
               Neuronal,Oligo,Microglia                848
               Oligo                                  1020
               Oligo,Microglia                         197
      

# are cell types specific elements enriched in plac-overlap? 

In [77]:
for x in ["glut-label", "gaba-label", "celltype_dif"]:
    """
    FET contigency table
                        Cell-type-active-label |  No cell-type-active-label
                        
                              -                       -
                              
    Plac-seq overlap.         A           |           B
    
    No Plac-seq overlap.      C           |           D
    
    """
    # first get the MPRA tiles that are cell-type specific 
    cell_ids = set(mpra.loc[mpra[x].astype(str)!="False", "name"])
    not_cell_ids = set(mpra.loc[mpra[x].astype(str)=="False", "name"])
   
    a = overlap.loc[overlap["name"].isin(cell_ids)].drop_duplicates().shape[0]  # in plac-seq, and is cell-specific
    b = overlap.loc[overlap["name"].isin(not_cell_ids)].drop_duplicates().shape[0] # in plac-seq, but not cell-specific
    c = no_overlap.loc[no_overlap["name"].isin(cell_ids)].drop_duplicates().shape[0]  # not in plac-seq, but cell-specific
    d = no_overlap.loc[no_overlap["name"].isin(not_cell_ids)].drop_duplicates().shape[0]  # not in plac-seq, not cell-specific

    table = [[a,b],[c,d]]

    print(x, table, stats.fisher_exact(table))

    """
    ###
    # Findings - 
    ###
    
    glut-label [[10699, 46884], [1151, 5528]] SignificanceResult(statistic=1.0960017333202579, pvalue=0.007284737459181158)
    gaba-label [[2702, 54881], [450, 6229]] SignificanceResult(statistic=0.6815051555992865, pvalue=2.679442914197887e-12)
    celltype_dif [[10416, 47167], [1209, 5470]] SignificanceResult(statistic=0.9991340099254531, pvalue=0.9732148983023949)
    
    (1) GLUT-active label is significantly enriched for plac-seq overlap (OR = 1.1, p=0.007)
    (2) GABA-active label is significantly depleted of plac-seq overlap (OR = 0.68, p=2.7e-12)
    (3) Cell-type different labels do NOT significantly more or less overlap PLAC-seq data compared with non-cell-type different labels 

    ###
    # problems with this analysis -
    ###
    
    (1) some regulatory elements are longer than others, more tiles, may inflate counts. Should do this at the gene regulatory element level
    (2) not-cell-active does not mean that the element in active. Could compare the background of all active elements instead of all elements tested. 
    (3) p-values are NOT FDR corrected for three tests 
    """

glut-label [[10699, 46884], [1151, 5528]] SignificanceResult(statistic=1.0960017333202579, pvalue=0.007284737459181158)
gaba-label [[2702, 54881], [450, 6229]] SignificanceResult(statistic=0.6815051555992865, pvalue=2.679442914197887e-12)
celltype_dif [[10416, 47167], [1209, 5470]] SignificanceResult(statistic=0.9991340099254531, pvalue=0.9732148983023949)


In [84]:
for x in ["glut-label", "gaba-label", "celltype_dif"]:
    """
    FET contigency table
                        Cell-type-active-label |  No cell-type-acivte-label
                        
                              -                       -
                              
    Yes annot Yes neuron Plac-seq overlap.         A           |           B
    
    No annot Yes neuron Plac-seq overlap.      C           |           D
    
    """
    # first get the MPRA tiles that are cell-type specific 
    cell_ids = set(mpra.loc[mpra[x].astype(str)!="False", "name"])
    not_cell_ids = set(mpra.loc[mpra[x].astype(str)=="False", "name"])

    for annot in ['h', "e", "p"]:
        in_annot = overlap.loc[(overlap["plac.datatype"] == annot)&
                                (overlap["plac.celltypes"].str.contains("Neuronal"))]
        out_annot = overlap.loc[(overlap["plac.datatype"] != annot)&
                                (overlap["plac.celltypes"].str.contains("Neuronal"))]
        
        a = in_annot.loc[in_annot["name"].isin(cell_ids)].drop_duplicates().shape[0]  # in plac-seq, and is cell-specific
        b = in_annot.loc[in_annot["name"].isin(not_cell_ids)].drop_duplicates().shape[0] # in plac-seq, but not cell-specific
        c = out_annot.loc[out_annot["name"].isin(cell_ids)].drop_duplicates().shape[0]  # not in plac-seq, but cell-specific
        d = out_annot.loc[out_annot["name"].isin(not_cell_ids)].drop_duplicates().shape[0]  # not in plac-seq, not cell-specific
    
        table = [[a,b],[c,d]]
    
        print(x, annot, table, stats.fisher_exact(table))

    """
    ###
    # Findings - 
    ###
    
    glut-label h [[3102, 13494], [4648, 19723]] SignificanceResult(statistic=0.9754565808173302, pvalue=0.33526272254339873)
    glut-label e [[1629, 7036], [6121, 26181]] SignificanceResult(statistic=0.9902825007413968, pvalue=0.7574192680655388)
    glut-label p [[3019, 12687], [4731, 20530]] SignificanceResult(statistic=1.0326191492124155, pvalue=0.21779753902161086)
    gaba-label h [[751, 15845], [1182, 23189]] SignificanceResult(statistic=0.9298485913932507, pvalue=0.12885519063317455)
    gaba-label e [[475, 8190], [1458, 30844]] SignificanceResult(statistic=1.226938737226803, pvalue=0.00020627796714688232)  #
    gaba-label p [[707, 14999], [1226, 24035]] SignificanceResult(statistic=0.9240825407936386, pvalue=0.10327103438823788)
    celltype_dif h [[2966, 13630], [4461, 19910]] SignificanceResult(statistic=0.9712126437603931, pvalue=0.2669135372542283)
    celltype_dif e [[1607, 7058], [5820, 26482]] SignificanceResult(statistic=1.0360054005155126, pvalue=0.25828207696784933)
    celltype_dif p [[2854, 12852], [4573, 20688]] SignificanceResult(statistic=1.0046170811789983, pvalue=0.8638810771099996)
        
    (1) GLUT-active label is NOT significantly enriched for neuron plac-seq overlap with any annotation
    (2) GABA-active label is significantly enriched for neuron enhancer plac-seq overlap (OR = 1.23, p=0.0002)
    (3) Cell-type different abel is NOT significantly enriched for neuron plac-seq overlap with any annotation

    ###
    # problems with this analysis -
    ###
    
    (1) some regulatory elements are longer than others, more tiles, may inflate counts. Should do this at the gene regulatory element level
    (2) not-cell-active does not mean that the element in active. Could compare the background of all active elements instead of all elements tested. 
    (3) p-values are NOT FDR corrected for three tests 
    """

glut-label h [[3102, 13494], [4648, 19723]] SignificanceResult(statistic=0.9754565808173302, pvalue=0.33526272254339873)
glut-label e [[1629, 7036], [6121, 26181]] SignificanceResult(statistic=0.9902825007413968, pvalue=0.7574192680655388)
glut-label p [[3019, 12687], [4731, 20530]] SignificanceResult(statistic=1.0326191492124155, pvalue=0.21779753902161086)
gaba-label h [[751, 15845], [1182, 23189]] SignificanceResult(statistic=0.9298485913932507, pvalue=0.12885519063317455)
gaba-label e [[475, 8190], [1458, 30844]] SignificanceResult(statistic=1.226938737226803, pvalue=0.00020627796714688232)
gaba-label p [[707, 14999], [1226, 24035]] SignificanceResult(statistic=0.9240825407936386, pvalue=0.10327103438823788)
celltype_dif h [[2966, 13630], [4461, 19910]] SignificanceResult(statistic=0.9712126437603931, pvalue=0.2669135372542283)
celltype_dif e [[1607, 7058], [5820, 26482]] SignificanceResult(statistic=1.0360054005155126, pvalue=0.25828207696784933)
celltype_dif p [[2854, 12852], [45

##### Other problems 
- pleiotropic elements? Oligo, microglia specific elements?
- lack of resolution in HiC category? Try Song? 
- Adult HiC, not Dev.